In [1]:
import pandas as pd

# ratings = pd.read_csv('C:/Users/adwiz/Documents/Courses/datascience_netology/datasets/ml-data/ratings.csv')
ratings = pd.read_csv('D:/Datasets/ml-latest-small/ratings.csv', usecols=['userId','movieId', 'rating'])
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [2]:
from scipy.sparse import coo_matrix
import numpy as np

user_item_matrix = coo_matrix((
    (ratings['rating'] >= 4).astype(np.float32),
    (ratings['userId'], ratings['movieId'])
))
user_item_matrix.eliminate_zeros()

In [3]:
# делим разреженную матрицу на обучающую и тестовую
total_len = user_item_matrix.data.size
train_len = int(total_len * .8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace=False)
train_mask = np.in1d(all_indices, train_indices)

In [4]:
def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            [arr.row[mask], arr.col[mask]]
        ),
        arr.shape
    )

In [5]:
train_csr = get_masked(user_item_matrix, train_mask).tocsr()
train = train_csr.T
user_items = user_item_matrix.T.tocsr()

test_coo = get_masked(user_item_matrix, ~train_mask)
test_csr = test_coo.tocsr()

In [6]:
import os
from implicit.als import  AlternatingLeastSquares
from implicit.approximate_als import AnnoyAlternatingLeastSquares

os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1' #UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
als = AlternatingLeastSquares(factors=10, iterations=10)
annoy = AnnoyAlternatingLeastSquares(factors=10, iterations=10)

In [7]:
%%time
als.fit(train)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: total: 1.16 s
Wall time: 199 ms


In [8]:
%%time
annoy.fit(train)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: total: 3.23 s
Wall time: 295 ms


In [9]:
users = list(set(test_coo.row))
small_users = users[:10000]

def get_recs(users, model):
    return {
        user: model.recommend(userid=user, user_items=user_items, N=50)
        for user in small_users
    }
user_item_matrix

<611x193610 sparse matrix of type '<class 'numpy.float32'>'
	with 48580 stored elements in COOrdinate format>

In [10]:
def hitrate(k, recs, users):
    hits = 0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user])
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
    return hits / len(users)

In [11]:
%%time
als_recs = get_recs(small_users, als)

ValueError: user_items must contain 1 row for every user in userids

In [12]:
%%time
print(hitrate(50, als_recs, small_users))

NameError: name 'als_recs' is not defined

In [13]:
%%time
anoy_recs = get_recs(small_users, annoy)

CPU times: total: 344 ms
Wall time: 340 ms


In [14]:
%%time
print(hitrate(50, anoy_recs, small_users))

ValueError: too many values to unpack (expected 2)